In [ ]:
# Structure-aware lysine accessibility scaffold (SASA + refined DAR)
# Optional installs:
# pip install biopython freesasa

import os, pandas as pd
from math import comb, exp

MODELS_DIR = 'tl1a_in_silico/assets/models'
os.makedirs(MODELS_DIR, exist_ok=True)

# If you have PDBs, place as tl1a_in_silico/assets/models/FabXX.pdb
# Otherwise, run your preferred modeler outside this scaffold.

try:
    import freesasa
except Exception:
    print('freesasa not installed; skipping SASA and using sequence-only fallback.')
    freesasa=None

rows=[]
if freesasa:
    for clone in FABS.keys():
        pdb_path = os.path.join(MODELS_DIR, f"{clone}.pdb")
        if not os.path.exists(pdb_path):
            continue
        structure = freesasa.Structure(pdb_path)
        result = freesasa.calc(structure)
        # residue-level SASA aggregate
        sasa_by_res = {}
        for i in range(structure.nAtoms()):
            resn = structure.residueName(i).strip()
            resid = (structure.chainLabel(i), structure.residueNumber(i))
            sasa_by_res.setdefault((resn,resid), 0.0)
            sasa_by_res[(resn,resid)] += result.atomArea(i)
        for (resn,(chain,resid)), area in sasa_by_res.items():
            if resn=='LYS':
                rows.append(dict(Clone=clone, Chain=chain, Resid=resid, SASA=round(area,2)))

if rows:
    df_sasa = pd.DataFrame(rows)
    df_sasa.to_csv('tl1a_in_silico/lys_sasa.csv', index=False)
    print('Wrote tl1a_in_silico/lys_sasa.csv')
else:
    df_sasa = pd.DataFrame(columns=['Clone','Chain','Resid','SASA'])

# CDR-based split; fallback if no SASA
K_rows=[]
for clone in FABS.keys():
    VH,VL=FABS[clone]['VH'],FABS[clone]['VL']
    H_cdr = CDRS[clone]['H1']+CDRS[clone]['H2']+CDRS[clone]['H3']
    L_cdr = CDRS[clone]['L1']+CDRS[clone]['L2']+CDRS[clone]['L3']
    total_K = VH.count('K')+VL.count('K')
    cdr_K = sum(s.count('K') for s in [H_cdr,L_cdr])
    fr_K  = total_K - cdr_K
    # accessible heuristic
    if not df_sasa.empty:
        acc_n = len(df_sasa[(df_sasa.Clone==clone) & (df_sasa.SASA>10)])
    else:
        acc_n = None
    K_FR, K_CDR = fr_K, cdr_K
    K_accessible = int(K_FR + 0.5*K_CDR)
    K_rows.append(dict(Clone=clone, K_FR=K_FR, K_CDR=K_CDR, K_accessible=K_accessible, SASA_hits=acc_n))

df_k = pd.DataFrame(K_rows).sort_values('Clone')
df_k.to_csv('tl1a_in_silico/lys_accessible_refined.csv', index=False)
print('Wrote tl1a_in_silico/lys_accessible_refined.csv')

# Refined DAR using K_accessible

def dar_stats(Kacc, eq, eff=0.45):
    p=1-exp(-eff*eq/max(1,Kacc))
    P=lambda k: comb(Kacc,k)*(p**k)*((1-p)**(Kacc-k))
    P12=sum(P(k) for k in (1,2))
    Pge4=sum(P(k) for k in range(4,Kacc+1))
    EDAR=sum(k*P(k) for k in range(Kacc+1))
    return round(P12,3), round(Pge4,3), round(EDAR,2)

rows=[]
for _,r in df_k.iterrows():
    best=None
    for eq in (4,5,6):
        P12,P4,ED = dar_stats(int(r.K_accessible), eq)
        obj=0.75*P12 - 0.25*abs(ED-1.5)
        ok = P4<=0.08
        key=(1 if ok else 0, obj)
        if best is None or key>best['key']:
            best={'Eq_best_refined':eq,'P_DAR_1_2_refined':P12,'P_DAR_ge4_refined':P4,'E_DAR_refined':ED,'key':key}
    rows.append(dict(Clone=r.Clone, **best))

df_ref = pd.DataFrame(rows)
df_ref.to_csv('tl1a_in_silico/dar_refined.csv', index=False)
print('Wrote tl1a_in_silico/dar_refined.csv')

# Quick conjugation risk table (add * if any CDR Lys likely exposed; coarse: CDR_K>0 and SASA_hits>0)
quick=[]
for _,r in df_k.iterrows():
    star = '*' if (r.K_CDR>0 and (r.SASA_hits if pd.notna(r.SASA_hits) else 0)>0) else ''
    m = df_ref[df_ref['Clone']==r.Clone].iloc[0] if not df_ref.empty and (df_ref['Clone']==r.Clone).any() else None
    quick.append(dict(Clone=r.Clone, K_FR=r.K_FR, K_CDR=r.K_CDR, K_accessible=r.K_accessible,
                      Eq_best_refined=(m['Eq_best_refined'] if m is not None else ''),
                      P_DAR_1_2_refined=(m['P_DAR_1_2_refined'] if m is not None else ''),
                      P_DAR_ge4_refined=(m['P_DAR_ge4_refined'] if m is not None else ''),
                      E_DAR_refined=(m['E_DAR_refined'] if m is not None else ''),
                      CDR_Lys_exposed=star))

pd.DataFrame(quick).sort_values('Clone').to_csv('tl1a_in_silico/dar_conjugation_quick.csv', index=False)
print('Wrote tl1a_in_silico/dar_conjugation_quick.csv')
